**Actividad - Autoencoder para detección de fraude**

In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    roc_auc_score, roc_curve, precision_recall_curve,
    average_precision_score
)
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from keras.models import Model
from keras.layers import Input, Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import warnings
warnings.filterwarnings('ignore')

if '__file__' in globals():
    ruta_base = os.path.dirname(os.path.abspath(__file__))
else:
    ruta_base = os.getcwd()

ruta_datos = os.path.join(ruta_base, "..", "data", "application_train.csv")
datos = pd.read_csv(ruta_datos)

filas, columnas = datos.shape
tasa_incumplimiento = datos["TARGET"].mean() * 100
casos_normales = (datos["TARGET"] == 0).sum()
casos_incumplidos = (datos["TARGET"] == 1).sum()

print(f"\nArchivo cargado correctamente: {filas} filas y {columnas} columnas")
print(f"Tasa de incumplimiento: {tasa_incumplimiento:.2f}%")
print(f"Casos normales: {casos_normales} | Casos incumplidos: {casos_incumplidos}")




Archivo cargado correctamente: 307511 filas y 122 columnas
Tasa de incumplimiento: 8.07%
Casos normales: 282686 | Casos incumplidos: 24825


In [11]:
# Selección de variables y creación de nuevas características

# Variables de tiempo
datos['EDAD'] = -datos['DAYS_BIRTH'] / 365
datos['AÑOS_EMPLEADO'] = -datos['DAYS_EMPLOYED'] / 365
datos['AÑOS_EMPLEADO'] = datos['AÑOS_EMPLEADO'].replace(1000.67, np.nan)
datos['DIAS_CAMBIO_TELEFONO'] = -datos['DAYS_LAST_PHONE_CHANGE']
datos['DIAS_PUBLICACION_ID'] = -datos['DAYS_ID_PUBLISH']

# Ratios financieros
datos['RATIO_CREDITO_INGRESO'] = datos['AMT_CREDIT'] / datos['AMT_INCOME_TOTAL']
datos['RATIO_ANUALIDAD_INGRESO'] = datos['AMT_ANNUITY'] / datos['AMT_INCOME_TOTAL']
datos['RATIO_ANUALIDAD_CREDITO'] = datos['AMT_ANNUITY'] / datos['AMT_CREDIT']
datos['RATIO_BIENES_CREDITO'] = datos['AMT_GOODS_PRICE'] / datos['AMT_CREDIT']
datos['INGRESO_PER_CAPITA'] = datos['AMT_INCOME_TOTAL'] / (datos['CNT_FAM_MEMBERS'] + 1)
datos['CREDITO_PER_CAPITA'] = datos['AMT_CREDIT'] / (datos['CNT_FAM_MEMBERS'] + 1)

# Detección de inconsistencias
datos['INCONS_SCORE_INGRESO'] = 0
mask1 = (datos['EXT_SOURCE_2'] < 0.3) & (datos['AMT_INCOME_TOTAL'] > datos['AMT_INCOME_TOTAL'].quantile(0.75))
datos.loc[mask1, 'INCONS_SCORE_INGRESO'] = 1

datos['INCONS_FAMILIA_INGRESO'] = 0
mask2 = (datos['CNT_FAM_MEMBERS'] >= 4) & (datos['AMT_INCOME_TOTAL'] < datos['AMT_INCOME_TOTAL'].quantile(0.25))
datos.loc[mask2, 'INCONS_FAMILIA_INGRESO'] = 1

columnas_docs = [col for col in datos.columns if 'FLAG_DOCUMENT' in col]
datos['TOTAL_DOCUMENTOS'] = datos[columnas_docs].sum(axis=1)
datos['INCONS_DOCS_CREDITO'] = 0
mask3 = (datos['TOTAL_DOCUMENTOS'] <= 2) & (datos['AMT_CREDIT'] > datos['AMT_CREDIT'].quantile(0.75))
datos.loc[mask3, 'INCONS_DOCS_CREDITO'] = 1

datos['TOTAL_INCONSISTENCIAS'] = (datos['INCONS_SCORE_INGRESO'] + 
                                   datos['INCONS_FAMILIA_INGRESO'] + 
                                   datos['INCONS_DOCS_CREDITO'])

# Variables de círculo social
datos['DEFAULTS_SOCIALES'] = (datos['DEF_30_CNT_SOCIAL_CIRCLE'].fillna(0) + 
                               datos['DEF_60_CNT_SOCIAL_CIRCLE'].fillna(0))
datos['OBSERVACIONES_SOCIALES'] = (datos['OBS_30_CNT_SOCIAL_CIRCLE'].fillna(0) + 
                                    datos['OBS_60_CNT_SOCIAL_CIRCLE'].fillna(0))
datos['TASA_DEFAULT_SOCIAL'] = datos['DEFAULTS_SOCIALES'] / (datos['OBSERVACIONES_SOCIALES'] + 1)
datos['TIENE_CIRCULO_RIESGOSO'] = (datos['DEFAULTS_SOCIALES'] > 0).astype(int)

# Variables de estabilidad
datos['TIENE_AUTO'] = (~datos['OWN_CAR_AGE'].isna()).astype(int)
datos['TIENE_PROPIEDAD'] = (datos['FLAG_OWN_REALTY'] == 'Y').astype(int)
datos['CAMBIO_TELEFONO_RECIENTE'] = (datos['DIAS_CAMBIO_TELEFONO'] < 180).astype(int)
datos['PUBLICACION_ID_RECIENTE'] = (datos['DIAS_PUBLICACION_ID'] < 365).astype(int)

datos['SCORE_ESTABILIDAD'] = (datos['TIENE_AUTO'] + datos['TIENE_PROPIEDAD'] + 
                               (1 - datos['CAMBIO_TELEFONO_RECIENTE']) + 
                               (1 - datos['PUBLICACION_ID_RECIENTE']))

# Interacciones y flags de riesgo
datos['RATIO_EDAD_EMPLEO'] = datos['EDAD'] / (datos['AÑOS_EMPLEADO'] + 1)
datos['SOBREENDEUDAMIENTO'] = (datos['RATIO_CREDITO_INGRESO'] > 8).astype(int)
datos['JOVEN_CREDITO_ALTO'] = ((datos['EDAD'] < 25) & (datos['AMT_CREDIT'] > 500000)).astype(int)
datos['EMPLEO_NUEVO_CREDITO_ALTO'] = ((datos['AÑOS_EMPLEADO'] < 0.5) & (datos['AMT_CREDIT'] > 300000)).astype(int)

columnas_bureau = [col for col in datos.columns if 'AMT_REQ_CREDIT_BUREAU' in col]
datos['TOTAL_CONSULTAS_BUREAU'] = datos[columnas_bureau].sum(axis=1)
datos['CONSULTAS_EXCESIVAS'] = (datos['TOTAL_CONSULTAS_BUREAU'] > 5).astype(int)

# Lista de variables seleccionadas
variables_seleccionadas = [
    'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY',
    'RATIO_CREDITO_INGRESO', 'RATIO_ANUALIDAD_INGRESO', 'RATIO_ANUALIDAD_CREDITO',
    'INGRESO_PER_CAPITA', 'CREDITO_PER_CAPITA',
    'EXT_SOURCE_2', 'EXT_SOURCE_3',
    'EDAD', 'AÑOS_EMPLEADO', 'DAYS_REGISTRATION', 'RATIO_EDAD_EMPLEO',
    'CNT_CHILDREN', 'CNT_FAM_MEMBERS', 'TASA_DEFAULT_SOCIAL', 'TIENE_CIRCULO_RIESGOSO',
    'REGION_RATING_CLIENT', 'REG_CITY_NOT_WORK_CITY',
    'TOTAL_DOCUMENTOS', 'SCORE_ESTABILIDAD',
    'TOTAL_INCONSISTENCIAS', 'INCONS_SCORE_INGRESO', 
    'INCONS_FAMILIA_INGRESO', 'INCONS_DOCS_CREDITO',
    'SOBREENDEUDAMIENTO', 'JOVEN_CREDITO_ALTO', 'EMPLEO_NUEVO_CREDITO_ALTO', 
    'CONSULTAS_EXCESIVAS', 'CAMBIO_TELEFONO_RECIENTE'
]

print(f"\nVariables creadas: {len(variables_seleccionadas)}")
print(f"Inconsistencias detectadas: {datos['TOTAL_INCONSISTENCIAS'].sum()}")


Variables creadas: 31
Inconsistencias detectadas: 89958


In [12]:
# PREPARACION DE DATOS

# Crear dataset de trabajo
datos_trabajo = datos[variables_seleccionadas + ['TARGET', 'SK_ID_CURR']].copy()

# Imputar valores nulos
for col in variables_seleccionadas:
    if datos_trabajo[col].isnull().sum() > 0:
        if col in ['EXT_SOURCE_2', 'EXT_SOURCE_3']:
            datos_trabajo[col].fillna(datos_trabajo[col].median(), inplace=True)
        else:
            datos_trabajo[col].fillna(0, inplace=True)

# Separar datos normales para entrenar el autoencoder
datos_normales = datos_trabajo[datos_trabajo['TARGET'] == 0].copy()
X_normales = datos_normales[variables_seleccionadas].values
X_todos = datos_trabajo[variables_seleccionadas].values
y_todos = datos_trabajo['TARGET'].values

# Dividir en entrenamiento y validacion
X_entrenamiento, X_validacion = train_test_split(X_normales, test_size=0.2, random_state=42)

# Escalar datos
escalador = RobustScaler()
X_entrenamiento_escalado = escalador.fit_transform(X_entrenamiento)
X_validacion_escalado = escalador.transform(X_validacion)
X_todos_escalado = escalador.transform(X_todos)

print(f"\nDatos preparados:")
print(f"Entrenamiento: {X_entrenamiento.shape}")
print(f"Validacion: {X_validacion.shape}")
print(f"Dataset completo: {X_todos.shape}")


Datos preparados:
Entrenamiento: (226148, 31)
Validacion: (56538, 31)
Dataset completo: (307511, 31)
